# 🔬 Skin Cancer Classification: Malignant vs Benign

This notebook trains a deep learning model to classify skin lesions as either **malignant** or **benign** using transfer learning.

**Dataset**: [Skin Cancer: Malignant vs. Benign](https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign)  
- 1800 benign images (224x224)
- 1800 malignant images (224x224)

---

## 📦 Step 1: Install & Import Libraries

In [ ]:
# Install kagglehub to download the dataset
!pip install kagglehub tensorflow matplotlib scikit-learn

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 📥 Step 2: Download the Dataset

In [ ]:
import kagglehub

# Download the skin cancer dataset
path = kagglehub.dataset_download("fanconic/skin-cancer-malignant-vs-benign")
print(f"Dataset downloaded to: {path}")

In [ ]:
# List contents to find the data structure
for root, dirs, files in os.walk(path):
    level = root.replace(path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files only
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... and {len(files) - 5} more files')

In [ ]:
# Set paths for train and test data
train_dir = os.path.join(path, 'train')
test_dir = os.path.join(path, 'test')

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")

# Count images in each class
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        count = len(os.listdir(class_path))
        print(f"Training - {class_name}: {count} images")

for class_name in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_name)
    if os.path.isdir(class_path):
        count = len(os.listdir(class_path))
        print(f"Testing - {class_name}: {count} images")

## 🖼️ Step 3: Visualize Sample Images

In [ ]:
# Visualize sample images from both classes
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

classes = ['benign', 'malignant']

for row, class_name in enumerate(classes):
    class_path = os.path.join(train_dir, class_name)
    images = os.listdir(class_path)[:5]
    
    for col, img_name in enumerate(images):
        img_path = os.path.join(class_path, img_name)
        img = plt.imread(img_path)
        axes[row, col].imshow(img)
        axes[row, col].axis('off')
        if col == 0:
            axes[row, col].set_title(class_name.upper(), fontsize=14, fontweight='bold')

plt.suptitle('Sample Images from Dataset', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## ⚙️ Step 4: Data Preprocessing & Augmentation

In [ ]:
# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# No augmentation for test data, only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"\nClass indices: {train_generator.class_indices}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Test samples: {test_generator.samples}")

## 🧠 Step 5: Build the Model (Transfer Learning with MobileNetV2)

In [ ]:
def build_model():
    """
    Build a transfer learning model using MobileNetV2.
    MobileNetV2 is efficient and works great for image classification.
    """
    # Load pre-trained MobileNetV2 without top layers
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model layers initially
    base_model.trainable = False
    
    # Add custom classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)  # Binary classification
    
    model = Model(inputs=base_model.input, outputs=output)
    
    return model, base_model

model, base_model = build_model()
model.summary()

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")

## 🚀 Step 6: Train the Model (Phase 1 - Train Top Layers)

In [ ]:
# Callbacks for better training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

# Phase 1: Train only the top layers
print("Phase 1: Training top layers only...")
print("="*50)

history1 = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

## 🔧 Step 7: Fine-tune the Model (Phase 2 - Unfreeze Top Layers)

In [ ]:
# Unfreeze the top layers of the base model for fine-tuning
base_model.trainable = True

# Freeze all layers except the last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Phase 2: Fine-tune
print("\nPhase 2: Fine-tuning top layers of base model...")
print("="*50)

history2 = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

## 📊 Step 8: Visualize Training History

In [ ]:
def plot_history(history1, history2):
    """Plot training and validation metrics."""
    # Combine histories
    acc = history1.history['accuracy'] + history2.history['accuracy']
    val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    phase1_epochs = len(history1.history['accuracy'])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy plot
    axes[0].plot(epochs, acc, 'b-', label='Training Accuracy', linewidth=2)
    axes[0].plot(epochs, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
    axes[0].axvline(x=phase1_epochs, color='g', linestyle='--', label='Fine-tuning Start')
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss plot
    axes[1].plot(epochs, loss, 'b-', label='Training Loss', linewidth=2)
    axes[1].plot(epochs, val_loss, 'r-', label='Validation Loss', linewidth=2)
    axes[1].axvline(x=phase1_epochs, color='g', linestyle='--', label='Fine-tuning Start')
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_history(history1, history2)

## 📈 Step 9: Evaluate on Test Set

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print(f"\n" + "="*50)
print(f"📊 TEST RESULTS")
print(f"="*50)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"="*50)

In [ ]:
# Get predictions for detailed analysis
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)
predicted_classes = (predictions > 0.5).astype(int).flatten()
true_classes = test_generator.classes
class_names = list(test_generator.class_indices.keys())

# Classification report
print("\n📋 CLASSIFICATION REPORT")
print("="*50)
print(classification_report(true_classes, predicted_classes, target_names=class_names))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            annot_kws={'size': 16})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)  # Recall for malignant
specificity = tn / (tn + fp)  # Recall for benign

print(f"\n📊 ADDITIONAL METRICS")
print(f"="*50)
print(f"True Positives (Malignant correctly identified): {tp}")
print(f"True Negatives (Benign correctly identified): {tn}")
print(f"False Positives (Benign misclassified as Malignant): {fp}")
print(f"False Negatives (Malignant misclassified as Benign): {fn}")
print(f"\nSensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")

## 🖼️ Step 10: Visualize Predictions

In [ ]:
# Visualize some predictions
def show_predictions(generator, predictions, num_images=10):
    """Display sample predictions with actual labels."""
    generator.reset()
    images, labels = next(generator)
    
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()
    
    class_names = ['Benign', 'Malignant']
    
    for i in range(num_images):
        ax = axes[i]
        ax.imshow(images[i])
        
        pred_prob = predictions[i][0]
        pred_class = 1 if pred_prob > 0.5 else 0
        true_class = int(labels[i])
        
        color = 'green' if pred_class == true_class else 'red'
        
        ax.set_title(
            f'Pred: {class_names[pred_class]} ({pred_prob:.2f})\n'
            f'True: {class_names[true_class]}',
            color=color,
            fontsize=10
        )
        ax.axis('off')
    
    plt.suptitle('Model Predictions (Green=Correct, Red=Incorrect)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

show_predictions(test_generator, predictions)

## 💾 Step 11: Save the Model

In [ ]:
# Save the model
model_save_path = 'skin_cancer_model.keras'
model.save(model_save_path)
print(f"✅ Model saved to: {model_save_path}")

# Also save as H5 format for compatibility
model.save('skin_cancer_model.h5')
print(f"✅ Model also saved as: skin_cancer_model.h5")

## 🔮 Step 12: Test with a Single Image

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_single_image(img_path, model, true_label=None):
    """
    Predict whether a single image shows benign or malignant skin cancer.
    
    Args:
        img_path: Path to the image file
        model: Trained model
        true_label: Optional - the actual label ('benign' or 'malignant') to check if prediction is correct
    
    Returns:
        Prediction result, confidence, and whether it was correct (if true_label provided)
    """
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    
    # Make prediction
    prediction = model.predict(img_array, verbose=0)[0][0]
    
    # Interpret result
    if prediction > 0.5:
        result = "MALIGNANT"
        confidence = prediction
    else:
        result = "BENIGN"
        confidence = 1 - prediction
    
    # Check if prediction is correct (if true_label provided)
    is_correct = None
    correctness_text = ""
    if true_label is not None:
        true_label_upper = true_label.upper()
        is_correct = (result == true_label_upper)
        if is_correct:
            correctness_text = "\n✅ CORRECT!"
            border_color = 'green'
        else:
            correctness_text = f"\n❌ WRONG! (Actual: {true_label_upper})"
            border_color = 'red'
    else:
        border_color = 'red' if result == 'MALIGNANT' else 'green'
    
    # Display
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(image.load_img(img_path))
    
    title_text = f'Prediction: {result}\nConfidence: {confidence:.2%}{correctness_text}'
    ax.set_title(title_text, fontsize=14, fontweight='bold', color=border_color)
    ax.axis('off')
    
    # Add colored border based on correctness
    for spine in ax.spines.values():
        spine.set_edgecolor(border_color)
        spine.set_linewidth(5)
        spine.set_visible(True)
    
    plt.tight_layout()
    plt.show()
    
    return result, confidence, is_correct

# Test with sample images from the test set (we KNOW the true labels!)
sample_malignant = os.path.join(test_dir, 'malignant', os.listdir(os.path.join(test_dir, 'malignant'))[0])
sample_benign = os.path.join(test_dir, 'benign', os.listdir(os.path.join(test_dir, 'benign'))[0])

print("="*50)
print("Testing with a MALIGNANT sample:")
print("="*50)
result, conf, correct = predict_single_image(sample_malignant, model, true_label='malignant')
print(f"Result: {result} ({conf:.2%} confidence)")
print(f"Correct: {'✅ YES' if correct else '❌ NO'}\n")

print("="*50)
print("Testing with a BENIGN sample:")
print("="*50)
result, conf, correct = predict_single_image(sample_benign, model, true_label='benign')
print(f"Result: {result} ({conf:.2%} confidence)")
print(f"Correct: {'✅ YES' if correct else '❌ NO'}")

## 📝 Summary

### What we did:
1. **Downloaded** the Skin Cancer dataset from Kaggle (3600 images total)
2. **Preprocessed** images with data augmentation to prevent overfitting
3. **Built** a transfer learning model using MobileNetV2
4. **Trained** in two phases:
   - Phase 1: Trained only the custom classification layers
   - Phase 2: Fine-tuned the top layers of MobileNetV2
5. **Evaluated** the model on the test set
6. **Saved** the model for future use

### Model Architecture:
- **Base**: MobileNetV2 (pre-trained on ImageNet)
- **Custom layers**: GlobalAveragePooling2D → Dense(256) → Dropout(0.5) → Dense(128) → Dropout(0.3) → Dense(1, sigmoid)

### To use the saved model:
```python
from tensorflow.keras.models import load_model
model = load_model('skin_cancer_model.keras')
# or
model = load_model('skin_cancer_model.h5')
```

---
⚠️ **Disclaimer**: This model is for educational purposes only and should NOT be used for actual medical diagnosis. Always consult a qualified dermatologist for skin cancer screening.